get df of all misclassified mike images
 for each image
    get all matchin set images
    identify the on directly priot to the misclasfied one
    

In [213]:
# source folder
C1_PROCESSED = r'C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\'

# destination folders
C1_SELECTED_TO = r'C:\DS Project\_resized\Images\resized_255_sq_swapped\tractors\Mike\to\\'
C1_SELECTED_FROM = r'C:\DS Project\_resized\Images\resized_255_sq_swapped\tractors\Mike\from\\'

# dataset for keras
C1_PREDICT = r'C:\DS Project\_for_keras_255_sq _swapped\mike\data\tractor\predict\\'


In [214]:
import psycopg2
import db_config as creds
import os
import time
import datetime
import shutil 


In [215]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
         
    conn = psycopg2.connect(conn_string)

    # Create a cursor object
    cursor = conn.cursor()
  
    #print ("connected!")
    return conn, cursor

def disconnect(conn,cursor):
    
    cursor.close()
    conn.close()
    #print ("disconnected!")

In [216]:
def findSwapInImg(orig_img,same_set):
    
    #list has been ordered by sql
    #print (same_set)
    #print ( same_set.index(orig_img))
    
    idx=0
    for set_img in same_set:
        
        #print (idx,set_img[0])
        
        if set_img[0] == orig_img:
    
            #find the index of the original image and then return the previous on
            try:
                #return the index before i.e. the photo take directly before
                #print (same_set[idx-1][0])
                return same_set[idx-1][0]
    
            except:
    
                return ""
    
        idx+=1
     

In [217]:
#get list of all misclassified images
sql_misclassified = "select i.filename, i.setid,i.datetaken_adjusted, i.direction_tl_255_sq, tr.true_direction from images i inner join images tr on i.filename = tr.filename where i.cameraid = 1 and i.direction_tl_255_sq != tr.true_direction"
sql_same_set = "select i.filename from images i where cameraid=1 and setid=%s order by i.filename asc;"


# Connecting to DB
conn, cursor = connect()

cursor.execute(sql_misclassified)
misclassified = cursor.fetchall() 

for row in misclassified:
    
    img_to_swap_out = row[0]
    true_direction = row[4]
    
    #print ("misclassified",img_to_swap_out, true_direction )
    
    #find all images in same set
    #cursor.execute(sql_same_set, str(row[1])
    cursor.execute(sql_same_set,(row[1],))
    same_set = cursor.fetchall() 
    
  
    # find a better images for the pool of other images
    img_to_swap_in = findSwapInImg(img_to_swap_out,same_set)
    print("") 
    print("Swap",img_to_swap_out, "for",img_to_swap_in)    
    print("")   
    
    #if a swap in img found 
    if img_to_swap_in:
        
        #need to:
        # 1. update the _for_keras prepared data by copying the new image into the predict folder and removing the old image
        #    cant just recreate the data set for keras as this will generate a whole new bunch of train and test subset images. 
        # 2. swap the image from the resized processed folder <> to the correct resized tractors/mike for or to folder just to keep the source data in synch
       
        ####
        ## 1.
        ####
        ##copy to _for_keras predict
        src = C1_PROCESSED + img_to_swap_in
        dest = C1_PREDICT + img_to_swap_in
        shutil.copy(src, dest) 
        print("copy into keras predict",src,dest)
        
        ##remove from for keras predict
        to_remove = C1_PREDICT + img_to_swap_out
        os.remove(to_remove)
        print("remove from keras predict",to_remove)
        print("")
        ####
        ## 2.
        ####
        #move new file into correct tractors folder
        new_file_to_move_from = C1_PROCESSED + img_to_swap_in
        old_file_to_move_to = C1_PROCESSED + img_to_swap_out
        
        if (true_direction == 1): # TO Images
            new_file_to_move_to = C1_SELECTED_TO + img_to_swap_in
            old_file_to_move_from = C1_SELECTED_TO + img_to_swap_out
        else:
            new_file_to_move_to = C1_SELECTED_FROM + img_to_swap_in
            old_file_to_move_from = C1_SELECTED_FROM + img_to_swap_out
           
        print ("move from processed to tractors ", new_file_to_move_from, new_file_to_move_to)
        shutil.move(new_file_to_move_from, new_file_to_move_to)
        
        #move out old file back into the pool
        print ("move from tractors back into processed ", old_file_to_move_from, old_file_to_move_to)
        shutil.move(old_file_to_move_from, old_file_to_move_to)          
        
    
  
    
    
    


Swap 04Mk0379.JPG for 04Mk0378.JPG

copy into keras predict C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\04Mk0378.JPG C:\DS Project\_for_keras_255_sq _swapped\mike\data\tractor\predict\\04Mk0378.JPG
remove from keras predict C:\DS Project\_for_keras_255_sq _swapped\mike\data\tractor\predict\\04Mk0379.JPG

move from processed to tractors  C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\04Mk0378.JPG C:\DS Project\_resized\Images\resized_255_sq_swapped\tractors\Mike\to\\04Mk0378.JPG
move from tractors back into processed  C:\DS Project\_resized\Images\resized_255_sq_swapped\tractors\Mike\to\\04Mk0379.JPG C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\04Mk0379.JPG

Swap 04Mk3679.JPG for 04Mk3678.JPG

copy into keras predict C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\04Mk3678.JPG C:\DS Project\_for_keras_255_sq _swapped\mike\data\tractor\predict\\04Mk3678.JPG
remove from keras predict C:\DS Project

copy into keras predict C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\04Mk3387.JPG C:\DS Project\_for_keras_255_sq _swapped\mike\data\tractor\predict\\04Mk3387.JPG
remove from keras predict C:\DS Project\_for_keras_255_sq _swapped\mike\data\tractor\predict\\04Mk3388.JPG

move from processed to tractors  C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\04Mk3387.JPG C:\DS Project\_resized\Images\resized_255_sq_swapped\tractors\Mike\from\\04Mk3387.JPG
move from tractors back into processed  C:\DS Project\_resized\Images\resized_255_sq_swapped\tractors\Mike\from\\04Mk3388.JPG C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\04Mk3388.JPG

Swap 04Mk3525.JPG for 04Mk3524.JPG

copy into keras predict C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\04Mk3524.JPG C:\DS Project\_for_keras_255_sq _swapped\mike\data\tractor\predict\\04Mk3524.JPG
remove from keras predict C:\DS Project\_for_keras_255_sq _swapped\mike\

copy into keras predict C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\03Mk2175.JPG C:\DS Project\_for_keras_255_sq _swapped\mike\data\tractor\predict\\03Mk2175.JPG
remove from keras predict C:\DS Project\_for_keras_255_sq _swapped\mike\data\tractor\predict\\03Mk2176.JPG

move from processed to tractors  C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\03Mk2175.JPG C:\DS Project\_resized\Images\resized_255_sq_swapped\tractors\Mike\to\\03Mk2175.JPG
move from tractors back into processed  C:\DS Project\_resized\Images\resized_255_sq_swapped\tractors\Mike\to\\03Mk2176.JPG C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\03Mk2176.JPG

Swap 03Mk1541.JPG for 03Mk1540.JPG

copy into keras predict C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\03Mk1540.JPG C:\DS Project\_for_keras_255_sq _swapped\mike\data\tractor\predict\\03Mk1540.JPG
remove from keras predict C:\DS Project\_for_keras_255_sq _swapped\mike\data

copy into keras predict C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\06Mk2884.JPG C:\DS Project\_for_keras_255_sq _swapped\mike\data\tractor\predict\\06Mk2884.JPG
remove from keras predict C:\DS Project\_for_keras_255_sq _swapped\mike\data\tractor\predict\\06Mk2885.JPG

move from processed to tractors  C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\06Mk2884.JPG C:\DS Project\_resized\Images\resized_255_sq_swapped\tractors\Mike\to\\06Mk2884.JPG
move from tractors back into processed  C:\DS Project\_resized\Images\resized_255_sq_swapped\tractors\Mike\to\\06Mk2885.JPG C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\06Mk2885.JPG

Swap 08Mk5439.JPG for 08Mk5438.JPG

copy into keras predict C:\DS Project\_resized\Images\resized_255_sq_swapped\processed\Mike\\08Mk5438.JPG C:\DS Project\_for_keras_255_sq _swapped\mike\data\tractor\predict\\08Mk5438.JPG
remove from keras predict C:\DS Project\_for_keras_255_sq _swapped\mike\data